In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **Percent Over 25 Completed High School**

In [6]:
over_25_completed_hs = pd.read_csv('/content/drive/My Drive/data/fatal-police-shootings-in-the-us/PercentOver25CompletedHighSchool.csv', encoding="windows-1252")
over_25_completed_hs.info()
over_25_completed_hs.head()
over_25_completed_hs.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Geographic Area       29329 non-null  object
 1   City                  29329 non-null  object
 2   percent_completed_hs  29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


,Geographic Area,City,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6


,Geographic Area,City,percent_completed_hs
count,29329,29329,29329
unique,51,24255,728
top,PA,Franklin city,100
freq,1762,16,1301


1. percent_completed_hs will need to be converted to numeric values
2. since the missing entries are minor compared to the total size of the dataset, we are just going to drop them**

In [7]:
print('Before: {} entries'.format(len(over_25_completed_hs)))
print('Entries with missing values: {}'.format(len(over_25_completed_hs[over_25_completed_hs['percent_completed_hs'] == '-'])))

# dropping rows with missing percent_completed_hs
over_25_completed_hs = over_25_completed_hs[over_25_completed_hs['percent_completed_hs'] != '-']
over_25_completed_hs['percent_completed_hs'] = over_25_completed_hs['percent_completed_hs'].astype('float64')
print('After: {} entries'.format(len(over_25_completed_hs)))

Before: 29329 entries
Entries with missing values: 197
After: 29132 entries


# **Percentage People Below Poverty Level**

In [8]:
people_below_poverty = pd.read_csv('/content/drive/My Drive/data/fatal-police-shootings-in-the-us/PercentagePeopleBelowPovertyLevel.csv', encoding="windows-1252")
people_below_poverty.info()
people_below_poverty.head()
people_below_poverty.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29329 entries, 0 to 29328
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29329 non-null  object
 1   City             29329 non-null  object
 2   poverty_rate     29329 non-null  object
dtypes: object(3)
memory usage: 687.5+ KB


,Geographic Area,City,poverty_rate
0,AL,Abanda CDP,78.8
1,AL,Abbeville city,29.1
2,AL,Adamsville city,25.5
3,AL,Addison town,30.7
4,AL,Akron town,42


,Geographic Area,City,poverty_rate
count,29329,29329,29329
unique,51,24255,771
top,PA,Franklin city,0
freq,1762,16,1464


1. poverty_rate will need to be converted to numeric values
2. since the missing entries are minor compared to the total size of the dataset, we are just going to drop them

In [9]:
print('Before: {} entries'.format(len(people_below_poverty)))
print('Entries with missing values: {}'.format(len(people_below_poverty[people_below_poverty['poverty_rate'] == '-'])))
people_below_poverty = people_below_poverty[people_below_poverty['poverty_rate'] != '-']
people_below_poverty['poverty_rate'] = people_below_poverty['poverty_rate'].astype('float64')
print('After: {} entries'.format(len(people_below_poverty)))

Before: 29329 entries
Entries with missing values: 201
After: 29128 entries


# **Median Household Income 2015**

In [10]:
median_household_income = pd.read_csv('/content/drive/My Drive/data/fatal-police-shootings-in-the-us/MedianHouseholdIncome2015.csv', encoding="windows-1252")
median_household_income.info()
median_household_income.head()
median_household_income.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29322 entries, 0 to 29321
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic Area  29322 non-null  object
 1   City             29322 non-null  object
 2   Median Income    29271 non-null  object
dtypes: object(3)
memory usage: 687.4+ KB


,Geographic Area,City,Median Income
0,AL,Abanda CDP,11207
1,AL,Abbeville city,25615
2,AL,Adamsville city,42575
3,AL,Addison town,37083
4,AL,Akron town,21667


,Geographic Area,City,Median Income
count,29322,29322,29271
unique,51,24249,14592
top,PA,Franklin city,(X)
freq,1762,16,1113




1.   '(X)', '-', and na will need to be removed
2.   median income will be converted to numeric values
3.   values 250000+ and 2500- are safe to drop since they are just outliers and causes issues with manipulating data.



In [11]:
print('Cleaning data...')
print('Before: {} entries'.format(len(median_household_income)))
median_household_income = median_household_income[median_household_income['Median Income'] != '(X)']
median_household_income = median_household_income[median_household_income['Median Income'] != '-']
median_household_income = median_household_income[median_household_income['Median Income'].notna()]

# drop 250000+ and 2500-
median_household_income = median_household_income[~median_household_income['Median Income'].str.contains('-')]
median_household_income = median_household_income[~median_household_income['Median Income'].str.contains('+', regex=False)]

median_household_income['Median Income'] = median_household_income['Median Income'].astype('float64')
print('After: {} entries'.format(len(median_household_income)))

Cleaning data...
Before: 29322 entries
After: 27385 entries


# **Share Race By City**

In [12]:
share_race_by_city = pd.read_csv('/content/drive/My Drive/data/fatal-police-shootings-in-the-us/ShareRaceByCity.csv', encoding="windows-1252")
share_race_by_city.info()
share_race_by_city.head()
share_race_by_city.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29268 entries, 0 to 29267
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Geographic area        29268 non-null  object
 1   City                   29268 non-null  object
 2   share_white            29268 non-null  object
 3   share_black            29268 non-null  object
 4   share_native_american  29268 non-null  object
 5   share_asian            29268 non-null  object
 6   share_hispanic         29268 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


,Geographic area,City,share_white,share_black,share_native_american,share_asian,share_hispanic
0,AL,Abanda CDP,67.2,30.2,0,0,1.6
1,AL,Abbeville city,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,99.1,0.1,0,0.1,0.4
4,AL,Akron town,13.2,86.5,0,0,0.3


,Geographic area,City,share_white,share_black,share_native_american,share_asian,share_hispanic
count,29268,29268,29268,29268,29268,29268,29268
unique,51,24219,997,926,628,409,956
top,PA,Franklin city,100,0,0,0,0
freq,1764,16,1051,6587,6930,8537,2489




1.   Dropping '(X)'
2.   Convert share_white, share_black, share_native_american, share_asian, share_hispanic to numeric values



In [13]:
print('Before: {} entries.'.format(len(share_race_by_city)))

share_race_by_city = share_race_by_city[share_race_by_city['share_white']!='(X)']
share_race_by_city['share_white'] = share_race_by_city['share_white'].astype('float64')

share_race_by_city['share_black'] = share_race_by_city['share_black'].astype('float64')
share_race_by_city['share_native_american'] = share_race_by_city['share_native_american'].astype('float64')
share_race_by_city['share_asian'] = share_race_by_city['share_asian'].astype('float64')
share_race_by_city['share_hispanic'] = share_race_by_city['share_hispanic'].astype('float64')

print('After: {} entries.'.format(len(share_race_by_city)))

Before: 29268 entries.
After: 29248 entries.


# **Police Killing US**

In [14]:
police_killing_us = pd.read_csv('/content/drive/My Drive/data/fatal-police-shootings-in-the-us/PoliceKillingsUS.csv', encoding="windows-1252")
police_killing_us.info()
police_killing_us.head()
police_killing_us.describe()
police_killing_us.describe(include='object')

FileNotFoundError: ignored



1.   Dropping na



In [ ]:
police_killing_us = police_killing_us.dropna()

# **Combining**

Creating 'demographic' dataset, which is the combination of 'people_below_poverty', 'median_household_income', 'share_race_by_city', and 'over_25_completed_hs'

In [ ]:
a = [people_below_poverty.set_index(['Geographic Area', 'City']), median_household_income.set_index(['Geographic Area', 'City']), share_race_by_city.set_index(['Geographic area', 'City'])]
demographic = over_25_completed_hs.set_index(['Geographic Area', 'City']).join(other=a).reset_index()
demographic.info()
demographic.head()
demographic.describe()
demographic.describe(include=['object'])

In [ ]:
police_killing_us.head()

In [ ]:
demographic.groupby(['Geographic Area']).size().sort_values()

In [ ]:
demographic[demographic['Geographic Area']=='RI']['City'].sort_values()

In [ ]:
police_killing_us[police_killing_us['state']=='RI']['city']

We can see that although the city naming convention in 'police_shooting_us' is different from 'demographic', they only have the suffix 'CDP' or 'city'. We are going to remove those.

In [ ]:
#### find the redundant name
demographic['City'].str.split().str[-1].value_counts()

In [ ]:
demographic['City'] = demographic['City'].str.replace(' city.*','')
demographic['City'] = demographic['City'].str.replace(' CDP.*','')
demographic['City'] = demographic['City'].str.replace(' town.*','')
#demographic['City'] = demographic['City'].str.replace(' borough.*','')
#demographic['City'] = demographic['City'].str.replace(' village.*','')

In [ ]:
# rename column name for demographic
demographic = demographic.rename(columns={'Geographic Area':'state', 'City':'city'})
demographic['city'].head()
#
fatal_police_shooting = demographic.set_index(['state','city']).join(police_killing_us.set_index(['state','city'])).reset_index()
fatal_police_shooting.head()
fatal_police_shooting.info()

In [ ]:
fatal_police_shooting
print(len(police_killing_us))

# **Output**
Eventhough we can merge all data into one single dataset, 'demographic' and 'police_killing_us' hold distinct contexts and can be analyze separately. Therefore, I'm going to output both out:
1.   demographic.csv
2.   police_klling_us.csv
3.   combined.csv (joined data between the other two)
4.   cleaned-MedianHouseholdIncome2015.csv
5.   cleaned-PercentOver25CompletedHighSchool.csv
6.   cleaned-PercentagePeopleBelowPoverty.csv
7.   cleaned-PoliceKillingsUS.csv
8.   cleaned-ShareRaceByCity.csv


In [ ]:
demographic.to_csv('/content/drive/My Drive/data/cleaned-data/demographic.csv', index=False)
police_killing_us.to_csv('/content/drive/My Drive/data/cleaned-data/police_killing_us.csv', index=False)
fatal_police_shooting.to_csv('/content/drive/My Drive/data/cleaned-data/combined.csv', index=False)
median_household_income.to_csv('/content/drive/My Drive/data/cleaned-data/cleaned-MedianHouseholdIncome2015.csv', index=False)
over_25_completed_hs.to_csv('/content/drive/My Drive/data/cleaned-data/cleaned-PercentOver25CompletedHighSchool.csv', index=False)
people_below_poverty.to_csv('/content/drive/My Drive/data/cleaned-data/cleaned-PercentagePeopleBelowPoverty.csv', index=False)
police_killing_us.to_csv('/content/drive/My Drive/data/cleaned-data/cleaned-PoliceKillingsUS.csv', index=False)
share_race_by_city.to_csv('/content/drive/My Drive/data/cleaned-data/cleaned-ShareRaceByCity.csv', index=False)